In [ ]:
# Step 1: Install dependencies
!pip install yfinance --quiet
!pip install tensorflow --quiet

# Step 2: Import libraries
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout

# Step 3: Load Google stock price data using yfinance
ticker = 'GOOG'
df = yf.download(ticker, start='2015-01-01', end='2024-12-31')
df = df[['Close']]
df.dropna(inplace=True)

plt.figure(figsize=(10, 4))
plt.plot(df['Close'])
plt.title(f'{ticker} Stock Price')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.grid(True)
plt.show()

# Step 4: Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# Step 5: Prepare time series sequences
def create_sequences(data, window_size):
    x, y = [], []
    for i in range(len(data) - window_size):
        x.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(x), np.array(y)

window_size = 60
x, y = create_sequences(scaled_data, window_size)

# Split into training and testing
split = int(0.8 * len(x))
x_train, y_train = x[:split], y[:split]
x_test, y_test = x[split:], y[split:]

print("Train shape:", x_train.shape, y_train.shape)
print("Test shape:", x_test.shape, y_test.shape)

# Step 6: Build RNN model
model = Sequential([
    SimpleRNN(50, activation='tanh', input_shape=(window_size, 1), return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# Step 7: Train the model
history = model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))

# Step 8: Evaluate the model
predicted = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted)
true_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

# Step 9: Plot predictions vs actual
plt.figure(figsize=(10, 5))
plt.plot(true_prices, label='Actual Prices')
plt.plot(predicted_prices, label='Predicted Prices')
plt.title(f'{ticker} Stock Price Prediction - RNN')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

# Step 10: Save model
model.save('goog_rnn_model.h5')
